In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Set random seed for reproducibility
torch.manual_seed(42)

def generataGaussian(tindx):
    return torch.normal(tindx[0],tindx[1],(1,42,42))

class MyDataSet(Dataset):
    def __init__(self,n):
        classes={0:(0.5,2),1:(1,2.5)}
        self.y=[torch.round(torch.rand(1))[0].long() for i in range(n)]
        self.x=[generateGaussian(classes[self.y[i].item()]) for i in range(n)]
    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]

    def __len__(self):
        return len(self.X)

# Define transforms and create data loaders
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define Modified CNN model architecture
class ModifiedCNN(nn.Module):
    def __init__(self):
        super(ModifiedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(32, 4, kernel_size=3, stride=2, padding=2)
        self.fc1 = nn.Linear(32 * 7 * 7, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the modified model
modified_model = ModifiedCNN()

# Count the number of learnable parameters in the modified model
modified_num_params = sum(p.numel() for p in modified_model.parameters() if p.requires_grad)
print("Number of learnable parameters in modified model:", modified_num_params)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modified_model.parameters(), lr=0.001)

# Training the Modified CNN model
num_epochs = 5
modified_accuracy_list = []
modified_num_params_list = []

for epoch in range(num_epochs):
    modified_model.train()
    running_loss = 0.0

    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = modified_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader)}")

    # Evaluate the modified model on the test set
    modified_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = modified_model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    modified_accuracy_list.append(accuracy)
    modified_num_params_list.append(modified_num_params)

# Plot the percentage drop in parameters vs accuracy
baseline_num_params = num_params
percentage_drop = [(baseline_num_params - num_params) / baseline_num_params * 100 for num_params in modified_num_params_list]

plt.figure(figsize=(10, 6))
plt.plot(percentage_drop, modified_accuracy_list, marker='o')
plt.title('Percentage Drop in Parameters vs Accuracy')
plt.xlabel('Percentage Drop in Parameters')
plt.ylabel('Accuracy (%)')
plt.grid(True)
plt.show()


Matplotlib is building the font cache; this may take a moment.


NameError: name 'Dataset' is not defined